## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [1]:
# import libraries
import pandas as pd
import fuzzywuzzy as fuzzy
from fuzzywuzzy import process

# Load up store_income_data.csv
store = pd.read_csv('store_income_data_task.csv')
store.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [2]:
# get unique values in country
countries = store['country'].unique()
print(f"There are {len(countries)} unique countries.")
print(countries)

# convert to lowercase
store['country'] = store['country'].str.lower()
# remove whitespace
store['country'] = store['country'].str.strip()
store['country'].unique()

There are 77 unique countries.
['United States/' 'Britain' ' United States' 'Britain/' ' United Kingdom'
 'U.K.' 'SA ' 'U.K/' 'America' 'United Kingdom' nan 'united states'
 ' S.A.' 'England ' 'UK' 'S.A./' 'ENGLAND' 'BRITAIN' 'U.K' 'U.K '
 'America/' 'SA.' 'S.A. ' 'u.k' 'uk' ' ' 'UK.' 'England/' 'england'
 ' Britain' 'united states of america' 'UK/' 'SA/' 'SA' 'England.'
 'UNITED KINGDOM' 'America.' 'S.A..' 's.a.' ' U.K'
 ' United States of America' 'Britain ' 'England' ' SA'
 'United States of America.' 'United States of America/' 'United States.'
 's. africasouth africa' ' England' 'United Kingdom '
 'United States of America ' ' UK' 'united kingdom' 'AMERICA' 'America '
 'UNITED STATES OF AMERICA' ' S. AfricaSouth Africa' 'america'
 'S. AFRICASOUTH AFRICA' 'Britain.' '/' 'United Kingdom.' 'United States'
 ' America' 'UNITED STATES' 'sa' 'United States of America' 'UK '
 'United States ' 'S. AfricaSouth Africa/' 'S.A.' 'United Kingdom/'
 'S. AfricaSouth Africa ' 'S. AfricaSouth Afric

array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [3]:
def replace_matching(data, column, string_to_match, min_ratio = 85):
    # get unique strings
    all_strings = data[column].unique()
    # find matches. used example as guide bcoz i'm still a bit confused about this.
    matches = fuzzy.process.extract(string_to_match, all_strings, limit=15, scorer=fuzzy.fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    # locate data to replace
    matched_rows = data[column].isin(close_matches)
    # replace strings
    data.loc[matched_rows, column] = string_to_match  
    print(f"All close matches for {string_to_match} fixed!")

# modify all close matches to different strings.
replace_matching(store, 'country', 'united kingdom')
replace_matching(store, 'country', 'south africa')
replace_matching(store, 'country', 'england')
replace_matching(store, 'country', 'america')
replace_matching(store, 'country', 'united states')
replace_matching(store, 'country', 'united states of america')
replace_matching(store, 'country', 'uk')
replace_matching(store, 'country', 'britain')
replace_matching(store, 'country', 's.a.')
replace_matching(store, 'country', 's. africa south africa')

All close matches for united kingdom fixed!
All close matches for south africa fixed!
All close matches for england fixed!
All close matches for america fixed!
All close matches for united states fixed!
All close matches for united states of america fixed!
All close matches for uk fixed!
All close matches for britain fixed!
All close matches for s.a. fixed!
All close matches for s. africa south africa fixed!


In [4]:
store['country'].unique()

array(['united states', 'britain', 'united kingdom', 'u.k.', 'sa', 'u.k/',
       'america', nan, 's.a.', 'england', 'uk', 'u.k', 'sa.', '',
       'united states of america', 'sa/', 's. africa south africa', '/',
       '.'], dtype=object)

In [5]:
# some of the entries were not resolved by fuzzy matching and had to be manually replaced e.g. 'u.k/'
store['country'].replace('britain', 'united kingdom', inplace = True)
store['country'].replace('england', 'united kingdom', inplace = True)
store['country'].replace('uk', 'united kingdom', inplace = True)
store['country'].replace('u.k.', 'united kingdom', inplace = True)
store['country'].replace('sa', 'south africa', inplace = True)
store['country'].replace('u.k/', 'united kingdom', inplace = True)
store['country'].replace('u.k', 'united kingdom', inplace = True)
store['country'].replace('america', 'united states', inplace = True)
store['country'].replace('s.a.', 'south africa', inplace = True)
store['country'].replace('united states of america', 'united states', inplace = True)
store['country'].replace('s. africa south africa', 'south africa', inplace = True)
store['country'].replace('/', 'unknown', inplace = True)
store['country'].replace('.', 'unknown', inplace = True)
store['country'].replace('', 'unknown', inplace = True)
store['country'].replace('sa.', 'south africa', inplace = True)
store['country'].replace('sa/', 'south africa', inplace = True)

# drop null and unknown values.
store.dropna(axis=0, inplace=True)
store.drop(store[store['country'] == 'unknown'].index, inplace=True)
print(store['country'].unique())

['united kingdom' 'south africa' 'united states']


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [6]:
# convert date measured column to datetime format
store['date_measured'] = pd.to_datetime(store['date_measured'], format="%d-%m-%Y") 
# do the math to calculate how long ago the entry was measured
days_ago = pd.to_datetime("now") - store['date_measured']
# create a new column to store difference in days. parse it using to_timedelta just to be sure it's all in the same format
store['days ago'] = pd.to_timedelta(days_ago)
# round off because we only want an integer for days. 
store['days ago'] = store['days ago'].round('d')
# see results
store.head()


,id,store_name,store_email,department,income,date_measured,country,days ago
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,2006-05-08,united kingdom,6520 days
5,6,"Auburn National Bancorporation, Inc.",ccaldeyroux5@dion.ne.jp,Grocery,$69798987.04,1999-09-19,united kingdom,8943 days
6,7,"Interlink Electronics, Inc.",orodenborch6@skyrock.com,Garden,$22521052.79,2001-06-08,south africa,8315 days
9,10,"Synopsys, Inc.",lcancellieri9@tmall.com,Electronics,$44091294.62,2006-07-11,united kingdom,6456 days
15,16,New Home Company Inc. (The),nhinchcliffef@whitehouse.gov,Shoes,$90808764.99,1993-04-21,united kingdom,11285 days


In [7]:
store.isna().sum()
# no nan values present in dataset. 

id               0
store_name       0
store_email      0
department       0
income           0
date_measured    0
country          0
days ago         0
dtype: int64